# Medical Data Processing and Document Analysis
This notebook processes company FAQs and medical data from Mayo Clinic for document analysis.

## 1. Environment Setup and Dependencies

In [1]:
# Import required libraries
import pandas as pd
import os
from dotenv import load_dotenv
from pathlib import Path
from huggingface_hub import login


# Embedding and Vector Store imports
from langchain.schema import Document
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    MarkdownHeaderTextSplitter,
)
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_openai import ChatOpenAI

import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

import time
from typing import List, Dict, Any, Tuple
import warnings
warnings.filterwarnings('ignore')


# Initialize environment
load_dotenv()
login(os.getenv("HUGGINGFACE_HUB_TOKEN"))

# Configure data directories
data_dir = Path('data')
company_info_dir = Path(data_dir/'company_info')

## 2. Data Loading and Processing
### 2.1 Company FAQ Processing

In [2]:
def load_faq_documents(faq_path: Path) -> List[Document]:
    """
    Load and process FAQ documents from CSV file.
    
    Args:
        faq_path: Path to the FAQ CSV file
        
    Returns:
        List of Document objects
    """
    try:
        # Validate file exists
        if not faq_path.exists():
            raise FileNotFoundError(f"FAQ file not found at {faq_path}")
            
        df = pd.read_csv(faq_path)
        
        # Validate required columns
        required_cols = ['Question', 'Answer']
        if not all(col in df.columns for col in required_cols):
            raise ValueError(f"CSV must contain columns: {required_cols}")
            
        documents = []
        for idx, row in df.iterrows():
            content = f"Question: {row.get('Question', '')}\nAnswer: {row.get('Answer', '')}"
            
            doc = Document(
                page_content=content,
                metadata={
                    "source": "company_faq",
                    "type": "faq", 
                    "doc_id": f"{idx}", 
                }
            )
            documents.append(doc)
            
        logger.info(f"Loaded {len(documents)} FAQ documents")
        return documents
        
    except Exception as e:
        logger.error(f"Error loading FAQ documents: {str(e)}")
        raise


### 2.2 Company Information File Processing

In [3]:
def load_company_info(info_path: Path) -> Document:
    """
    Load company information from markdown file.
    
    Args:
        info_path: Path to the company info markdown file
        
    Returns:
        Document object containing company info
    """
    try:
        # Validate file exists
        if not info_path.exists():
            raise FileNotFoundError(f"Info file not found at {info_path}")
            
        with open(info_path, 'r', encoding='utf-8') as f:
            content = f.read()
            
        doc = Document(
            page_content=content,
            metadata={
                "source": "company_info",
                "type": "general_info",
                "filename": info_path.name,
                "doc_id": "company_info_main"
            }
        )
        logger.info("Loaded company info document")
        return doc
        
    except Exception as e:
        logger.error(f"Error loading company info: {str(e)}")
        raise


In [13]:
# Load all documents
faq_path = '..' / company_info_dir / 'FAQ.csv'
info_path = '..' / company_info_dir / "info.md"

company_documents = load_faq_documents(faq_path)
company_documents.append(load_company_info(info_path))

print(f"✅ Loaded {len(company_documents)} company documents")
company_documents[-5:]

INFO:__main__:Loaded 299 FAQ documents
INFO:__main__:Loaded company info document


✅ Loaded 300 company documents


[Document(metadata={'source': 'company_faq', 'type': 'faq', 'doc_id': '295'}, page_content='Question: هل يمكنني الحصول على استشارة بخصوص مشاكل الجهاز المناعي؟\nAnswer: نعم، يمكن لأطبائنا تقديم استشارات أولية حول مشاكل الجهاز المناعي.'),
 Document(metadata={'source': 'company_faq', 'type': 'faq', 'doc_id': '296'}, page_content='Question: هل توفرون استشارات بخصوص مشاكل ضعف التركيز؟\nAnswer: نعم، يمكن لأخصائيي الرعاية النفسية أو الأعصاب تقديم استشارات حول مشاكل التركيز.'),
 Document(metadata={'source': 'company_faq', 'type': 'faq', 'doc_id': '297'}, page_content='Question: هل يمكنني الحصول على استشارة بخصوص صحة العظام والهشاشة؟\nAnswer: نعم، يمكن لأطبائنا تقديم استشارات حول صحة العظام والوقاية من الهشاشة.'),
 Document(metadata={'source': 'company_faq', 'type': 'faq', 'doc_id': '298'}, page_content='Question: هل توفرون استشارات لأمراض الكلى المزمنة؟\nAnswer: نعم، يمكن لأطبائنا تقديم استشارات أولية ومتابعة لمرضى الكلى المزمنة.'),
 Document(metadata={'source': 'company_info', 'type': 'genera

## 3. Text Splitting Configuration and Processing

In [14]:
recursive_500 = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""]
)


markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=[
        ("#", "company_title"),
        ("##", "section"),
    ]
)

# Process company documents
company_chunks = []

for doc in company_documents:
    if doc.metadata.get("type") == "general_info":
        # Use markdown splitter for info.md
        split_docs = markdown_splitter.split_text(doc.page_content)
        for d in split_docs:
            d.metadata.update(doc.metadata)
        company_chunks.extend(split_docs)
    else:
        # Use recursive splitter for FAQs
        split_docs = recursive_500.split_documents([doc])
        company_chunks.extend(split_docs)

print(f"Company chunks: {len(company_chunks)}")

Company chunks: 311


### 3.1 Evaluation of Document Chunk Quality

In [22]:
for i, doc in enumerate(company_chunks[-25:]):
    print(f"\n--- Chunk {i+1} ---")
    print("Metadata:", doc.metadata)
    print("content:\n",doc.page_content)


--- Chunk 1 ---
Metadata: {'source': 'company_faq', 'type': 'faq', 'doc_id': '286'}
content:
 Question: هل توفرون خدمات استشارية للطب الشرعي؟
Answer: لا، لا نقدم خدمات الطب الشرعي.

--- Chunk 2 ---
Metadata: {'source': 'company_faq', 'type': 'faq', 'doc_id': '287'}
content:
 Question: هل يمكنني الحصول على استشارة بخصوص الأرق واضطرابات النوم؟
Answer: نعم، يمكن لأخصائيي الرعاية النفسية تقديم استشارات حول الأرق واضطرابات النوم.

--- Chunk 3 ---
Metadata: {'source': 'company_faq', 'type': 'faq', 'doc_id': '288'}
content:
 Question: هل توفرون استشارات للأمراض العصبية؟
Answer: نعم، لدينا أطباء أعصاب يمكنهم تقديم استشارات أولية حول الأمراض العصبية.

--- Chunk 4 ---
Metadata: {'source': 'company_faq', 'type': 'faq', 'doc_id': '289'}
content:
 Question: هل يمكنني الحصول على استشارة بخصوص فقر الدم؟
Answer: نعم، يمكن لأطبائنا تقديم استشارات حول فقر الدم وأسبابه وعلاجه.

--- Chunk 5 ---
Metadata: {'source': 'company_faq', 'type': 'faq', 'doc_id': '290'}
content:
 Question: هل توفرون استشارات للطب

## 4. Testing Embedding models

### 4.1 Define Retrieval Functions

In [13]:
# Create vector retriever
def create_vector_retriever(documents: List[Document], embedding_model: str, k: int = 5):
   """
   Create a vector-based retriever.
   
   Args:
       documents: List of documents to index
       embedding_model: Name of the embedding model
       k: Number of documents to retrieve
   
   Returns:
       Tuple of (vector_retriever, vector_store)
   """
   embeddings = HuggingFaceEmbeddings(
       model_name=embedding_model,
       model_kwargs={'device': 'cpu'},
       encode_kwargs={'normalize_embeddings': True}
   )
   
   # Create vector store
   vector_store = FAISS.from_documents(documents, embeddings)
   vector_retriever = vector_store.as_retriever(search_kwargs={"k": k})
   
   return vector_retriever, vector_store

# Create BM25 retriever
def create_bm25_retriever(documents: List[Document], k: int = 5):
   """
   Create a BM25-based retriever.
   
   Args:
       documents: List of documents to index
       k: Number of documents to retrieve
   
   Returns:
       BM25Retriever object
   """
   bm25_retriever = BM25Retriever.from_documents(documents)
   bm25_retriever.k = k
   
   return bm25_retriever

# Create ensemble (hybrid) retriever
def create_hybrid_retriever(bm25_retriever, vector_retriever, weights: List[float] = [0.5, 0.5]):
   """
   Create a hybrid retriever combining BM25 and vector search.
   
   Args:
       bm25_retriever: BM25 retriever object
       vector_retriever: Vector retriever object
       weights: List of weights for [BM25, Vector] retrievers
   
   Returns:
       EnsembleRetriever object
   """
   ensemble_retriever = EnsembleRetriever(
       retrievers=[bm25_retriever, vector_retriever],
       weights=weights
   )

   return ensemble_retriever

print("🔧 Retrieval functions defined!")

🔧 Retrieval functions defined!


### 4.2 Evaluation for All Models and Retriever Types

In [25]:
def evaluate_retriever(retriever, queries: List[str], retriever_type: str) -> Dict[str, Any]:
    """
    Evaluate a retriever on given queries.

    Args:
        retriever: The retriever to evaluate
        queries: List of queries to test
        retriever_type: Type of retriever ('vector' or 'hybrid')

    Returns:
        Dict containing evaluation results
    """
    results = []
    total_time = 0

    for i, query in enumerate(queries):
        start_time = time.time()

        # Retrieve documents
        retrieved_docs = retriever.get_relevant_documents(query)

        end_time = time.time()
        query_time = end_time - start_time
        total_time += query_time

        # Store results
        query_result = {
            'query_id': i + 1,
            'query': query,
            'retriever_type': retriever_type,
            'num_retrieved': len(retrieved_docs),
            'retrieval_time': query_time,
            'retrieved_docs': [
                {
                    'content': doc.page_content[:500] + "..." if len(doc.page_content) > 500 else doc.page_content,  
                    'metadata': doc.metadata,
                    'relevance_score': getattr(doc, 'metadata', {}).get('score', 'N/A')
                }
                for doc in retrieved_docs[:10]
            ]
        }
        results.append(query_result)

    return {
        'results': results,
        'total_time': total_time,
        'avg_time_per_query': total_time / len(queries) if queries else 0
    }


# Embedding models to test
embedding_models = [
    "sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
    "intfloat/multilingual-e5-small",
    "ibm-granite/granite-embedding-107m-multilingual"
]

# Test queries 
test_queries = [
    "ما هي الخدمات الرئيسية التي تقدمونها",      # What are your main services?
    "كيف يمكنني التواصل معكم",                   # How can I contact you?
    "ما هي أوقات العمل",                        # What are your working hours?
]

k = 5


print("🎯 Configuration loaded:")
print(f"📊 Embedding models: {len(embedding_models)}")
print(f"🏢 Company queries: {len(test_queries)}")
print(f"Using k = {k} 'Number of retrieved documents':")

# Run Evaluation for All Models and Retriever Types
evaluation_results = {}

print("🚀 Starting evaluation of embedding models with different retriever types...\n")

for model_name in embedding_models:
    print(f"📊 Evaluating: {model_name}")
    print("-" * 80)

    model_start_time = time.time()

    try:
        # Create vector retriever
        print("🔍 Creating vector retriever...")
        vector_retriever, vector_store = create_vector_retriever(
            company_chunks, model_name, k=k
        )
        
        # Create BM25 retriever
        print("📝 Creating BM25 retriever...")
        bm25_retriever = create_bm25_retriever(company_chunks, k=k)
        
        # Create hybrid retriever
        print("🔄 Creating hybrid retriever...")
        hybrid_retriever = create_hybrid_retriever(
            bm25_retriever, vector_retriever, weights=[0.3, 0.7]
        )

        # Evaluate vector-only retriever
        print("🔍 Evaluating vector retriever...")
        vector_results = evaluate_retriever(vector_retriever, test_queries, "vector")

        # Evaluate hybrid retriever
        print("🔄 Evaluating hybrid retriever...")
        hybrid_results = evaluate_retriever(hybrid_retriever, test_queries, "hybrid")

        model_end_time = time.time()
        total_model_time = model_end_time - model_start_time

        # Store results
        evaluation_results[model_name] = {
            'vector_results': vector_results,
            'hybrid_results': hybrid_results,
            'total_time': total_model_time,
            'vector_avg_time': vector_results['avg_time_per_query'],
            'hybrid_avg_time': hybrid_results['avg_time_per_query']
        }

        print(f"✅ Completed in {total_model_time:.2f} seconds")
        print(f"🔍 Vector avg: {vector_results['avg_time_per_query']:.3f}s per query")
        print(f"🔄 Hybrid avg: {hybrid_results['avg_time_per_query']:.3f}s per query")

    except Exception as e:
        print(f"❌ Error with {model_name}: {str(e)}")
        evaluation_results[model_name] = {'error': str(e)}

    print("\n")

print("🎉 Evaluation completed for all models and retriever types!")

# Cell 6: Display Detailed Results
def display_results():
    """Display detailed results in a professional format."""

    print("=" * 100)
    print("📊 EMBEDDING MODELS & RETRIEVER TYPES EVALUATION RESULTS")
    print("=" * 100)

    # Summary table
    summary_data = []

    for model_name, results in evaluation_results.items():
        if 'error' not in results:
            summary_data.append({
                'Model': model_name.split('/')[-1],  # Short name
                'Total Time (s)': f"{results['total_time']:.2f}",
                'Vector Avg (s)': f"{results['vector_avg_time']:.3f}",
                'Hybrid Avg (s)': f"{results['hybrid_avg_time']:.3f}",
                'Queries Tested': len(results['vector_results']['results'])
            })

    summary_df = pd.DataFrame(summary_data)
    print("\n📈 PERFORMANCE SUMMARY")
    print("-" * 70)
    print(summary_df.to_string(index=False))

    # Detailed results for each model
    for model_name, results in evaluation_results.items():
        if 'error' not in results:
            print(f"\n🔍 DETAILED RESULTS: {model_name}")
            print("-" * 80)

            # Vector retriever results
            print("\n🔍 VECTOR RETRIEVER RESULTS:")
            for result in results['vector_results']['results']:
                print(f"\nQuery {result['query_id']}: {result['query']}")
                print(f"⏱️  Retrieval time: {result['retrieval_time']:.3f}s")
                print(f"📄 Retrieved {result['num_retrieved']} documents:")

                for i, doc in enumerate(result['retrieved_docs'], 1):  
                    print(f"  {i}. {doc['content']}")
                    print(f"     📁 Source: {doc['metadata'].get('source', 'N/A')}")

            # Hybrid retriever results
            print(f"\n🔄 HYBRID RETRIEVER RESULTS:")
            for result in results['hybrid_results']['results']:
                print(f"\nQuery {result['query_id']}: {result['query']}")
                print(f"⏱️  Retrieval time: {result['retrieval_time']:.3f}s")
                print(f"📄 Retrieved {result['num_retrieved']} documents:")

                for i, doc in enumerate(result['retrieved_docs'], 1):  
                    print(f"  {i}. {doc['content']}")
                    print(f"     📁 Source: {doc['metadata'].get('source', 'N/A')}")

# Display results
display_results()


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/paraphrase-multilingual-mpnet-base-v2


🎯 Configuration loaded:
📊 Embedding models: 3
🏢 Company queries: 3
Using k = 5 'Number of retrieved documents':
🚀 Starting evaluation of embedding models with different retriever types...

📊 Evaluating: sentence-transformers/paraphrase-multilingual-mpnet-base-v2
--------------------------------------------------------------------------------
🔍 Creating vector retriever...


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: intfloat/multilingual-e5-small


📝 Creating BM25 retriever...
🔄 Creating hybrid retriever...
🔍 Evaluating vector retriever...
🔄 Evaluating hybrid retriever...
✅ Completed in 9.55 seconds
🔍 Vector avg: 0.030s per query
🔄 Hybrid avg: 0.029s per query


📊 Evaluating: intfloat/multilingual-e5-small
--------------------------------------------------------------------------------
🔍 Creating vector retriever...


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: ibm-granite/granite-embedding-107m-multilingual


📝 Creating BM25 retriever...
🔄 Creating hybrid retriever...
🔍 Evaluating vector retriever...
🔄 Evaluating hybrid retriever...
✅ Completed in 7.51 seconds
🔍 Vector avg: 0.014s per query
🔄 Hybrid avg: 0.015s per query


📊 Evaluating: ibm-granite/granite-embedding-107m-multilingual
--------------------------------------------------------------------------------
🔍 Creating vector retriever...
📝 Creating BM25 retriever...
🔄 Creating hybrid retriever...
🔍 Evaluating vector retriever...
🔄 Evaluating hybrid retriever...
✅ Completed in 6.87 seconds
🔍 Vector avg: 0.008s per query
🔄 Hybrid avg: 0.007s per query


🎉 Evaluation completed for all models and retriever types!
📊 EMBEDDING MODELS & RETRIEVER TYPES EVALUATION RESULTS

📈 PERFORMANCE SUMMARY
----------------------------------------------------------------------
                                Model Total Time (s) Vector Avg (s) Hybrid Avg (s)  Queries Tested
paraphrase-multilingual-mpnet-base-v2           9.55          0.030          0.029

### 4.3 LLM Evaluation 

In [34]:
def prepare_llm_evaluation_prompt():
    """Prepare comprehensive prompt for LLM evaluation with retrieved documents analysis."""

    prompt = """
# Embedding Models and Retriever Types Evaluation Report for Company RAG System

I have tested 3 multilingual embedding models with 2 different retriever types for a company information retrieval system. Please analyze the results including the retrieved documents' relevance and recommend the best embedding model and the best retriever type.

## Tested Embedding Models:
1. sentence-transformers/paraphrase-multilingual-mpnet-base-v2
2. intfloat/multilingual-e5-small
3. ibm-granite/granite-embedding-107m-multilingual

## Tested Retriever Types:
1. **Vector Retriever:** Uses vector search only
2. **Hybrid Retriever:** Combines BM25 and vector search (Weights: 30% BM25, 70% Vector)

## Test Queries with Retrieved Documents Analysis:
"""

    for i, query in enumerate(test_queries, 1):
        prompt += f"\n### Query {i}: \"{query}\"\n"
        
        # Add retrieved documents for each model and retriever type
        for model_name, results in evaluation_results.items():
            if 'error' not in results:
                prompt += f"\n**{model_name}:**\n"
                
                # Vector retriever results for this query
                vector_result = results['vector_results']['results'][i-1]
                prompt += f"\n*Vector Retriever (Time: {vector_result['retrieval_time']:.3f}s):*\n"
                for j, doc in enumerate(vector_result['retrieved_docs'], 1):
                    prompt += f"{j}. \"{doc['content'][:300]}{'...' if len(doc['content']) > 300 else ''}\"\n"
                    prompt += f"   Source: {doc['metadata'].get('source', 'N/A')}\n"
                
                # Hybrid retriever results for this query
                hybrid_result = results['hybrid_results']['results'][i-1]
                prompt += f"\n*Hybrid Retriever (Time: {hybrid_result['retrieval_time']:.3f}s):*\n"
                for j, doc in enumerate(hybrid_result['retrieved_docs'], 1):
                    prompt += f"{j}. \"{doc['content'][:300]}{'...' if len(doc['content']) > 300 else ''}\"\n"
                    prompt += f"   Source: {doc['metadata'].get('source', 'N/A')}\n"

    prompt += "\n## Performance Summary:\n\n"

    # Add performance data summary
    for model_name, results in evaluation_results.items():
        if 'error' not in results:
            prompt += f"""
### {model_name}
- **Total Processing Time:** {results['total_time']:.2f} seconds
- **Vector Retriever Average Time:** {results['vector_avg_time']:.3f} seconds per query
- **Hybrid Retriever Average Time:** {results['hybrid_avg_time']:.3f} seconds per query
"""

    prompt += """

## Evaluation Criteria:
1. **Performance Speed:** Faster retrieval times
2. **Document Relevance:** How well retrieved documents match the query intent
3. **Multilingual Support:** Better handling of Arabic queries
4. **Result Quality:** Accuracy and relevance of retrieved documents
5. **Consistency:** Stable performance and relevance across different queries

## What I Need From You:
1. **Rank the Embedding Models:** (1st, 2nd, 3rd) with justification based on both speed AND document relevance
2. **Compare Retriever Types:** Which is better - Vector or Hybrid? Consider both speed and relevance
3. **Document Relevance Analysis:** For each query, which model+retriever combination returned the most relevant documents?
4. **Final Recommendation:** Best combination (embedding model + retriever type) considering all factors
5. **Detailed Analysis:** Strengths and weaknesses of each combination in terms of:
   - Speed performance
   - Document relevance quality
   - Consistency across queries
6. **Production Considerations:** Recommendations for production environment usage

## Objective:
Select the best combination that provides the optimal balance between:
- Response speed
- Quality and relevance of retrieved results
- Efficiency in processing Arabic text
- Performance stability
- Document-query matching accuracy

## Additional Context:
- The system will be used for company information retrieval
- Arabic language support is crucial
- Both speed and accuracy are important
- The system should return highly relevant documents
- Consider scalability for production use

## Analysis Instructions:
Please analyze each query's results and evaluate:
1. Which retrieved documents are most relevant to each query
2. Which model consistently returns better relevant documents
3. Whether vector or hybrid retrieval provides better document relevance
4. The trade-off between speed and relevance quality

Please structure your response with clear sections for:
1. Document Relevance Analysis by Query
2. Speed Performance Analysis
3. Overall Model Ranking
4. Retriever Type Comparison
5. Final Recommendation with Justification
"""

    return prompt

# Prepare the evaluation prompt
llm_prompt = prepare_llm_evaluation_prompt()
print("📝 LLM evaluation prompt prepared!")
print(f"📏 Prompt length: {len(llm_prompt)} characters")
display(Markdown(llm_prompt))

📝 LLM evaluation prompt prepared!
📏 Prompt length: 28937 characters



# Embedding Models and Retriever Types Evaluation Report for Company RAG System

I have tested 3 multilingual embedding models with 2 different retriever types for a company information retrieval system. Please analyze the results including the retrieved documents' relevance and recommend the best embedding model and the best retriever type.

## Tested Embedding Models:
1. sentence-transformers/paraphrase-multilingual-mpnet-base-v2
2. intfloat/multilingual-e5-small
3. ibm-granite/granite-embedding-107m-multilingual

## Tested Retriever Types:
1. **Vector Retriever:** Uses vector search only
2. **Hybrid Retriever:** Combines BM25 and vector search (Weights: 30% BM25, 70% Vector)

## Test Queries with Retrieved Documents Analysis:

### Query 1: "ما هي الخدمات الرئيسية التي تقدمونها"

**sentence-transformers/paraphrase-multilingual-mpnet-base-v2:**

*Vector Retriever (Time: 0.031s):*
1. "Question: ما هي الخدمات الرئيسية التي تقدمونها؟
Answer: نقدم الاستشارات الطبية عبر الفيديو، مراجعة التقارير الطبية، خدمات الصيدلة والتوصيل المنزلي، حجز مواعيد مع أطباء متخصصين، خدمات الرعاية النفسية، وتحليل نتائج المختبر."
   Source: company_faq
2. "Question: هل توفرون خدمة مخصصة لكبار السن؟
Answer: نسعى لتلبية احتياجات جميع الفئات العمرية، ونرحب بالملاحظات لتحسين الخدمة لكبار السن."
   Source: company_faq
3. "Question: هل يمكنني تقديم ملاحظات أو اقتراحات لتحسين الخدمة؟
Answer: نعم، نشجع بشدة تقديم الملاحظات والاقتراحات، فهي تساعدنا على تحسين خدماتنا باستمرار."
   Source: company_faq
4. "Question: هل توفرون خدمات استشارية للطب الباطني العام؟
Answer: نعم، لدينا أطباء باطنية عامون للتعامل مع مجموعة واسعة من الحالات."
   Source: company_faq
5. "Question: هل توفرون خدمة الدعم عبر الواتساب؟
Answer: نعم، قد نوفر خدمة الدعم عبر الواتساب لبعض الاستفسارات السريعة. يرجى مراجعة موقعنا لمعرفة الرقم."
   Source: company_faq

*Hybrid Retriever (Time: 0.029s):*
1. "Question: ما هي الخدمات الرئيسية التي تقدمونها؟
Answer: نقدم الاستشارات الطبية عبر الفيديو، مراجعة التقارير الطبية، خدمات الصيدلة والتوصيل المنزلي، حجز مواعيد مع أطباء متخصصين، خدمات الرعاية النفسية، وتحليل نتائج المختبر."
   Source: company_faq
2. "Question: هل توفرون خدمة مخصصة لكبار السن؟
Answer: نسعى لتلبية احتياجات جميع الفئات العمرية، ونرحب بالملاحظات لتحسين الخدمة لكبار السن."
   Source: company_faq
3. "Question: هل يمكنني تقديم ملاحظات أو اقتراحات لتحسين الخدمة؟
Answer: نعم، نشجع بشدة تقديم الملاحظات والاقتراحات، فهي تساعدنا على تحسين خدماتنا باستمرار."
   Source: company_faq
4. "Question: هل توفرون خدمات استشارية للطب الباطني العام؟
Answer: نعم، لدينا أطباء باطنية عامون للتعامل مع مجموعة واسعة من الحالات."
   Source: company_faq
5. "Question: هل توفرون خدمة الدعم عبر الواتساب؟
Answer: نعم، قد نوفر خدمة الدعم عبر الواتساب لبعض الاستفسارات السريعة. يرجى مراجعة موقعنا لمعرفة الرقم."
   Source: company_faq
6. "Question: ما هي الرؤية الرئيسية لشركة الشفاء الرقمية؟
Answer: رؤيتنا أن نكون الخيار الأول في العالم العربي للرعاية الصحية الرقمية، ونحدث فرقًا حقيقيًا في جودة حياة الأفراد."
   Source: company_faq
7. "Question: ما هي القيم الأساسية للشركة؟
Answer: قيمنا الأساسية هي السرية والخصوصية، الجودة والاحترافية، التيسير والسرعة، والموثوقية والشفافية."
   Source: company_faq
8. "Question: ما هي رسالة الشفاء الرقمية؟
Answer: رسالتنا هي تقديم خدمات طبية موثوقة، آمنة، وسهلة الوصول لجميع أفراد المجتمع من خلال التكنولوجيا الحديثة."
   Source: company_faq
9. "Question: ما نوع الأجهزة التي أحتاجها للاستشارة عبر الفيديو؟
Answer: تحتاج إلى جهاز كمبيوتر أو هاتف ذكي/جهاز لوحي متصل بالإنترنت وبه كاميرا وميكروفون."
   Source: company_faq

**intfloat/multilingual-e5-small:**

*Vector Retriever (Time: 0.015s):*
1. "Question: ما هي الخدمات الرئيسية التي تقدمونها؟
Answer: نقدم الاستشارات الطبية عبر الفيديو، مراجعة التقارير الطبية، خدمات الصيدلة والتوصيل المنزلي، حجز مواعيد مع أطباء متخصصين، خدمات الرعاية النفسية، وتحليل نتائج المختبر."
   Source: company_faq
2. "تقديم خدمات طبية **موثوقة، آمنة، وسهلة الوصول** لجميع أفراد المجتمع، مع التركيز على الكفاءة المهنية والشفافية التامة، وذلك من خلال تسخير **أحدث التكنولوجيا الرقمية** لبناء جسر فعال بين المرضى والخبراء الطبيين. نلتزم بتقديم تجربة رعاية صحية سلسة وشاملة، تراعي احتياجات كل فرد وتضمن خصوصيته وأمن معلوما..."
   Source: company_info
3. "نقدم في شركة الشفاء الرقمية مجموعة واسعة ومتكاملة من الخدمات الصحية الرقمية التي تلبي مختلف احتياجات الرعاية:  
* **الاستشارات الطبية عبر الفيديو (Video Consultations):**
* استشارات فورية ومجدولة مع أطباء عامين ومتخصصين في مختلف التخصصات (الباطنية، الأطفال، الجلدية، النساء والولادة، العيون، الأنف وا..."
   Source: company_info
4. "Question: هل توفرون خدمة مخصصة لكبار السن؟
Answer: نسعى لتلبية احتياجات جميع الفئات العمرية، ونرحب بالملاحظات لتحسين الخدمة لكبار السن."
   Source: company_faq
5. "في شركة الشفاء الرقمية، تتجسد قيمنا في كل جانب من جوانب عملنا، وهي تشكل حجر الزاوية الذي نبني عليه علاقاتنا مع مرضانا وشركائنا:  
* **السرية والخصوصية (Confidentiality & Privacy):** نلتزم بأقصى درجات السرية في التعامل مع بيانات المرضى ومعلوماتهم الصحية، ونطبق بروتوكولات أمان صارمة لضمان حماية معلوما..."
   Source: company_info

*Hybrid Retriever (Time: 0.015s):*
1. "Question: ما هي الخدمات الرئيسية التي تقدمونها؟
Answer: نقدم الاستشارات الطبية عبر الفيديو، مراجعة التقارير الطبية، خدمات الصيدلة والتوصيل المنزلي، حجز مواعيد مع أطباء متخصصين، خدمات الرعاية النفسية، وتحليل نتائج المختبر."
   Source: company_faq
2. "تقديم خدمات طبية **موثوقة، آمنة، وسهلة الوصول** لجميع أفراد المجتمع، مع التركيز على الكفاءة المهنية والشفافية التامة، وذلك من خلال تسخير **أحدث التكنولوجيا الرقمية** لبناء جسر فعال بين المرضى والخبراء الطبيين. نلتزم بتقديم تجربة رعاية صحية سلسة وشاملة، تراعي احتياجات كل فرد وتضمن خصوصيته وأمن معلوما..."
   Source: company_info
3. "نقدم في شركة الشفاء الرقمية مجموعة واسعة ومتكاملة من الخدمات الصحية الرقمية التي تلبي مختلف احتياجات الرعاية:  
* **الاستشارات الطبية عبر الفيديو (Video Consultations):**
* استشارات فورية ومجدولة مع أطباء عامين ومتخصصين في مختلف التخصصات (الباطنية، الأطفال، الجلدية، النساء والولادة، العيون، الأنف وا..."
   Source: company_info
4. "Question: هل توفرون خدمة مخصصة لكبار السن؟
Answer: نسعى لتلبية احتياجات جميع الفئات العمرية، ونرحب بالملاحظات لتحسين الخدمة لكبار السن."
   Source: company_faq
5. "في شركة الشفاء الرقمية، تتجسد قيمنا في كل جانب من جوانب عملنا، وهي تشكل حجر الزاوية الذي نبني عليه علاقاتنا مع مرضانا وشركائنا:  
* **السرية والخصوصية (Confidentiality & Privacy):** نلتزم بأقصى درجات السرية في التعامل مع بيانات المرضى ومعلوماتهم الصحية، ونطبق بروتوكولات أمان صارمة لضمان حماية معلوما..."
   Source: company_info
6. "Question: ما هي الرؤية الرئيسية لشركة الشفاء الرقمية؟
Answer: رؤيتنا أن نكون الخيار الأول في العالم العربي للرعاية الصحية الرقمية، ونحدث فرقًا حقيقيًا في جودة حياة الأفراد."
   Source: company_faq
7. "Question: ما هي القيم الأساسية للشركة؟
Answer: قيمنا الأساسية هي السرية والخصوصية، الجودة والاحترافية، التيسير والسرعة، والموثوقية والشفافية."
   Source: company_faq
8. "Question: ما هي رسالة الشفاء الرقمية؟
Answer: رسالتنا هي تقديم خدمات طبية موثوقة، آمنة، وسهلة الوصول لجميع أفراد المجتمع من خلال التكنولوجيا الحديثة."
   Source: company_faq
9. "Question: ما نوع الأجهزة التي أحتاجها للاستشارة عبر الفيديو؟
Answer: تحتاج إلى جهاز كمبيوتر أو هاتف ذكي/جهاز لوحي متصل بالإنترنت وبه كاميرا وميكروفون."
   Source: company_faq

**ibm-granite/granite-embedding-107m-multilingual:**

*Vector Retriever (Time: 0.010s):*
1. "Question: ما هي الخدمات الرئيسية التي تقدمونها؟
Answer: نقدم الاستشارات الطبية عبر الفيديو، مراجعة التقارير الطبية، خدمات الصيدلة والتوصيل المنزلي، حجز مواعيد مع أطباء متخصصين، خدمات الرعاية النفسية، وتحليل نتائج المختبر."
   Source: company_faq
2. "Question: هل توفرون خدمات الرعاية الصحية المنزلية؟
Answer: حالياً، خدماتنا تركز على الرعاية الصحية عن بعد، ولا نوفر زيارات منزلية مباشرة."
   Source: company_faq
3. "Question: هل توفرون خدمة الإسعافات الأولية عن بعد؟
Answer: لا، لا نقدم خدمة الإسعافات الأولية عن بعد، فهي تتطلب تدخلاً مباشراً."
   Source: company_faq
4. "Question: هل توفرون خدمات استشارية للرعاية المنزلية لكبار السن؟
Answer: لا نقدم خدمات الرعاية المنزلية المباشرة، ولكن يمكننا تقديم المشورة العامة حولها."
   Source: company_faq
5. "تقديم خدمات طبية **موثوقة، آمنة، وسهلة الوصول** لجميع أفراد المجتمع، مع التركيز على الكفاءة المهنية والشفافية التامة، وذلك من خلال تسخير **أحدث التكنولوجيا الرقمية** لبناء جسر فعال بين المرضى والخبراء الطبيين. نلتزم بتقديم تجربة رعاية صحية سلسة وشاملة، تراعي احتياجات كل فرد وتضمن خصوصيته وأمن معلوما..."
   Source: company_info

*Hybrid Retriever (Time: 0.006s):*
1. "Question: ما هي الخدمات الرئيسية التي تقدمونها؟
Answer: نقدم الاستشارات الطبية عبر الفيديو، مراجعة التقارير الطبية، خدمات الصيدلة والتوصيل المنزلي، حجز مواعيد مع أطباء متخصصين، خدمات الرعاية النفسية، وتحليل نتائج المختبر."
   Source: company_faq
2. "Question: هل توفرون خدمات الرعاية الصحية المنزلية؟
Answer: حالياً، خدماتنا تركز على الرعاية الصحية عن بعد، ولا نوفر زيارات منزلية مباشرة."
   Source: company_faq
3. "Question: هل توفرون خدمة الإسعافات الأولية عن بعد؟
Answer: لا، لا نقدم خدمة الإسعافات الأولية عن بعد، فهي تتطلب تدخلاً مباشراً."
   Source: company_faq
4. "Question: هل توفرون خدمات استشارية للرعاية المنزلية لكبار السن؟
Answer: لا نقدم خدمات الرعاية المنزلية المباشرة، ولكن يمكننا تقديم المشورة العامة حولها."
   Source: company_faq
5. "تقديم خدمات طبية **موثوقة، آمنة، وسهلة الوصول** لجميع أفراد المجتمع، مع التركيز على الكفاءة المهنية والشفافية التامة، وذلك من خلال تسخير **أحدث التكنولوجيا الرقمية** لبناء جسر فعال بين المرضى والخبراء الطبيين. نلتزم بتقديم تجربة رعاية صحية سلسة وشاملة، تراعي احتياجات كل فرد وتضمن خصوصيته وأمن معلوما..."
   Source: company_info
6. "Question: ما هي الرؤية الرئيسية لشركة الشفاء الرقمية؟
Answer: رؤيتنا أن نكون الخيار الأول في العالم العربي للرعاية الصحية الرقمية، ونحدث فرقًا حقيقيًا في جودة حياة الأفراد."
   Source: company_faq
7. "Question: ما هي القيم الأساسية للشركة؟
Answer: قيمنا الأساسية هي السرية والخصوصية، الجودة والاحترافية، التيسير والسرعة، والموثوقية والشفافية."
   Source: company_faq
8. "Question: ما هي رسالة الشفاء الرقمية؟
Answer: رسالتنا هي تقديم خدمات طبية موثوقة، آمنة، وسهلة الوصول لجميع أفراد المجتمع من خلال التكنولوجيا الحديثة."
   Source: company_faq
9. "Question: ما نوع الأجهزة التي أحتاجها للاستشارة عبر الفيديو؟
Answer: تحتاج إلى جهاز كمبيوتر أو هاتف ذكي/جهاز لوحي متصل بالإنترنت وبه كاميرا وميكروفون."
   Source: company_faq

### Query 2: "كيف يمكنني التواصل معكم"

**sentence-transformers/paraphrase-multilingual-mpnet-base-v2:**

*Vector Retriever (Time: 0.029s):*
1. "Question: كيف يمكنني التواصل مع الدعم الفني؟
Answer: يمكنك التواصل مع الدعم الفني عبر الدردشة المباشرة، الهاتف (9200-000-000)، أو البريد الإلكتروني (support@alshifa-care.com)."
   Source: company_faq
2. "Question: ما هي الإجراءات المتبعة في حال انقطاع الاتصال أثناء الاستشارة؟
Answer: في حال انقطاع الاتصال، سيحاول الطبيب أو فريق الدعم إعادة الاتصال بك لإكمال الاستشارة."
   Source: company_faq
3. "Question: هل يمكنني التواصل مع الطبيب بعد انتهاء الاستشارة؟
Answer: يمكن التواصل مع الطبيب للمتابعة ضمن استشارة جديدة، أو للاستفسارات القصيرة عبر خاصية الرسائل إن كانت متاحة في باقتك."
   Source: company_faq
4. "نحن هنا لخدمتكم والإجابة على استفساراتكم. يمكنكم التواصل معنا عبر القنوات التالية:  
* **الهاتف:** 9200-000-000 (متوفر خلال ساعات العمل الرسمية)
* **البريد الإلكتروني:** support@alshifa-care.com (نعد بالرد على استفساراتكم خلال 24 ساعة عمل)
* **الموقع الإلكتروني:** [www.alshifa-care.com](https://www...."
   Source: company_info
5. "Question: هل يمكنني تلقي إشعارات على جوالي؟
Answer: نعم، يمكنك تفعيل الإشعارات على تطبيق الجوال لتلقي التنبيهات والتذكيرات."
   Source: company_faq

*Hybrid Retriever (Time: 0.029s):*
1. "Question: كيف يمكنني التواصل مع الدعم الفني؟
Answer: يمكنك التواصل مع الدعم الفني عبر الدردشة المباشرة، الهاتف (9200-000-000)، أو البريد الإلكتروني (support@alshifa-care.com)."
   Source: company_faq
2. "Question: ما هي الإجراءات المتبعة في حال انقطاع الاتصال أثناء الاستشارة؟
Answer: في حال انقطاع الاتصال، سيحاول الطبيب أو فريق الدعم إعادة الاتصال بك لإكمال الاستشارة."
   Source: company_faq
3. "Question: هل يمكنني التواصل مع الطبيب بعد انتهاء الاستشارة؟
Answer: يمكن التواصل مع الطبيب للمتابعة ضمن استشارة جديدة، أو للاستفسارات القصيرة عبر خاصية الرسائل إن كانت متاحة في باقتك."
   Source: company_faq
4. "نحن هنا لخدمتكم والإجابة على استفساراتكم. يمكنكم التواصل معنا عبر القنوات التالية:  
* **الهاتف:** 9200-000-000 (متوفر خلال ساعات العمل الرسمية)
* **البريد الإلكتروني:** support@alshifa-care.com (نعد بالرد على استفساراتكم خلال 24 ساعة عمل)
* **الموقع الإلكتروني:** [www.alshifa-care.com](https://www...."
   Source: company_info
5. "Question: هل يمكنني تلقي إشعارات على جوالي؟
Answer: نعم، يمكنك تفعيل الإشعارات على تطبيق الجوال لتلقي التنبيهات والتذكيرات."
   Source: company_faq
6. "Question: كيف يمكنني إلغاء الاشتراك؟
Answer: يمكنك تسجيل الدخول إلى حسابك والتوجه إلى "إعدادات الحساب" ثم "إلغاء الاشتراك"، أو التواصل مع الدعم الفني."
   Source: company_faq
7. "Question: كيف يمكنني الاشتراك في الخدمة؟
Answer: قم بإنشاء حساب في موقعنا، اختر الباقة المناسبة، وأدخل معلومات الدفع."
   Source: company_faq
8. "Question: كيف يمكنني التأكد من كفاءة الأطباء؟
Answer: يتم اختيار الأطباء بعناية فائقة بناءً على مؤهلاتهم، خبرتهم، وتراخيصهم المهنية، ويتم تقييم أدائهم باستمرار."
   Source: company_faq
9. "Question: هل توفرون خدمة التوعية الصحية عبر وسائل التواصل الاجتماعي؟
Answer: نعم، ننشر محتوى توعوياً بشكل منتظم على صفحاتنا الرسمية في وسائل التواصل الاجتماعي."
   Source: company_faq

**intfloat/multilingual-e5-small:**

*Vector Retriever (Time: 0.013s):*
1. "Question: كيف يمكنني التواصل مع الدعم الفني؟
Answer: يمكنك التواصل مع الدعم الفني عبر الدردشة المباشرة، الهاتف (9200-000-000)، أو البريد الإلكتروني (support@alshifa-care.com)."
   Source: company_faq
2. "Question: هل يمكنني التواصل مع الطبيب بعد انتهاء الاستشارة؟
Answer: يمكن التواصل مع الطبيب للمتابعة ضمن استشارة جديدة، أو للاستفسارات القصيرة عبر خاصية الرسائل إن كانت متاحة في باقتك."
   Source: company_faq
3. "نحن هنا لخدمتكم والإجابة على استفساراتكم. يمكنكم التواصل معنا عبر القنوات التالية:  
* **الهاتف:** 9200-000-000 (متوفر خلال ساعات العمل الرسمية)
* **البريد الإلكتروني:** support@alshifa-care.com (نعد بالرد على استفساراتكم خلال 24 ساعة عمل)
* **الموقع الإلكتروني:** [www.alshifa-care.com](https://www...."
   Source: company_info
4. "Question: هل يمكنني الحصول على مشورة بخصوص اللقاحات؟
Answer: نعم، يمكن لأطبائنا تقديم مشورة حول اللقاحات الموصى بها والجدول الزمني لها."
   Source: company_faq
5. "Question: كيف يمكنني إلغاء الاشتراك؟
Answer: يمكنك تسجيل الدخول إلى حسابك والتوجه إلى "إعدادات الحساب" ثم "إلغاء الاشتراك"، أو التواصل مع الدعم الفني."
   Source: company_faq

*Hybrid Retriever (Time: 0.017s):*
1. "Question: كيف يمكنني التواصل مع الدعم الفني؟
Answer: يمكنك التواصل مع الدعم الفني عبر الدردشة المباشرة، الهاتف (9200-000-000)، أو البريد الإلكتروني (support@alshifa-care.com)."
   Source: company_faq
2. "Question: كيف يمكنني إلغاء الاشتراك؟
Answer: يمكنك تسجيل الدخول إلى حسابك والتوجه إلى "إعدادات الحساب" ثم "إلغاء الاشتراك"، أو التواصل مع الدعم الفني."
   Source: company_faq
3. "Question: هل يمكنني التواصل مع الطبيب بعد انتهاء الاستشارة؟
Answer: يمكن التواصل مع الطبيب للمتابعة ضمن استشارة جديدة، أو للاستفسارات القصيرة عبر خاصية الرسائل إن كانت متاحة في باقتك."
   Source: company_faq
4. "نحن هنا لخدمتكم والإجابة على استفساراتكم. يمكنكم التواصل معنا عبر القنوات التالية:  
* **الهاتف:** 9200-000-000 (متوفر خلال ساعات العمل الرسمية)
* **البريد الإلكتروني:** support@alshifa-care.com (نعد بالرد على استفساراتكم خلال 24 ساعة عمل)
* **الموقع الإلكتروني:** [www.alshifa-care.com](https://www...."
   Source: company_info
5. "Question: هل يمكنني الحصول على مشورة بخصوص اللقاحات؟
Answer: نعم، يمكن لأطبائنا تقديم مشورة حول اللقاحات الموصى بها والجدول الزمني لها."
   Source: company_faq
6. "Question: كيف يمكنني الاشتراك في الخدمة؟
Answer: قم بإنشاء حساب في موقعنا، اختر الباقة المناسبة، وأدخل معلومات الدفع."
   Source: company_faq
7. "Question: كيف يمكنني التأكد من كفاءة الأطباء؟
Answer: يتم اختيار الأطباء بعناية فائقة بناءً على مؤهلاتهم، خبرتهم، وتراخيصهم المهنية، ويتم تقييم أدائهم باستمرار."
   Source: company_faq
8. "Question: هل توفرون خدمة التوعية الصحية عبر وسائل التواصل الاجتماعي؟
Answer: نعم، ننشر محتوى توعوياً بشكل منتظم على صفحاتنا الرسمية في وسائل التواصل الاجتماعي."
   Source: company_faq

**ibm-granite/granite-embedding-107m-multilingual:**

*Vector Retriever (Time: 0.007s):*
1. "Question: كيف يمكنني التواصل مع الدعم الفني؟
Answer: يمكنك التواصل مع الدعم الفني عبر الدردشة المباشرة، الهاتف (9200-000-000)، أو البريد الإلكتروني (support@alshifa-care.com)."
   Source: company_faq
2. "Question: هل يمكنني التواصل مع الطبيب بعد انتهاء الاستشارة؟
Answer: يمكن التواصل مع الطبيب للمتابعة ضمن استشارة جديدة، أو للاستفسارات القصيرة عبر خاصية الرسائل إن كانت متاحة في باقتك."
   Source: company_faq
3. "Question: هل يمكنني الاستفسار عن مدى تفاعل دواءين مع بعضهما؟
Answer: نعم، يمكن للطبيب أو الصيدلي تقديم معلومات حول التفاعلات الدوائية المحتملة."
   Source: company_faq
4. "Question: كيف يمكنني إلغاء الاشتراك؟
Answer: يمكنك تسجيل الدخول إلى حسابك والتوجه إلى "إعدادات الحساب" ثم "إلغاء الاشتراك"، أو التواصل مع الدعم الفني."
   Source: company_faq
5. "Question: هل يمكنني التواصل مع خدمة العملاء خارج أوقات العمل؟
Answer: الدردشة المباشرة متوفرة 24/7، أما الدعم الهاتفي والبريد الإلكتروني فيكون خلال ساعات العمل المحددة."
   Source: company_faq

*Hybrid Retriever (Time: 0.009s):*
1. "Question: كيف يمكنني التواصل مع الدعم الفني؟
Answer: يمكنك التواصل مع الدعم الفني عبر الدردشة المباشرة، الهاتف (9200-000-000)، أو البريد الإلكتروني (support@alshifa-care.com)."
   Source: company_faq
2. "Question: كيف يمكنني إلغاء الاشتراك؟
Answer: يمكنك تسجيل الدخول إلى حسابك والتوجه إلى "إعدادات الحساب" ثم "إلغاء الاشتراك"، أو التواصل مع الدعم الفني."
   Source: company_faq
3. "Question: هل يمكنني التواصل مع الطبيب بعد انتهاء الاستشارة؟
Answer: يمكن التواصل مع الطبيب للمتابعة ضمن استشارة جديدة، أو للاستفسارات القصيرة عبر خاصية الرسائل إن كانت متاحة في باقتك."
   Source: company_faq
4. "Question: هل يمكنني الاستفسار عن مدى تفاعل دواءين مع بعضهما؟
Answer: نعم، يمكن للطبيب أو الصيدلي تقديم معلومات حول التفاعلات الدوائية المحتملة."
   Source: company_faq
5. "Question: هل يمكنني التواصل مع خدمة العملاء خارج أوقات العمل؟
Answer: الدردشة المباشرة متوفرة 24/7، أما الدعم الهاتفي والبريد الإلكتروني فيكون خلال ساعات العمل المحددة."
   Source: company_faq
6. "Question: كيف يمكنني الاشتراك في الخدمة؟
Answer: قم بإنشاء حساب في موقعنا، اختر الباقة المناسبة، وأدخل معلومات الدفع."
   Source: company_faq
7. "Question: كيف يمكنني التأكد من كفاءة الأطباء؟
Answer: يتم اختيار الأطباء بعناية فائقة بناءً على مؤهلاتهم، خبرتهم، وتراخيصهم المهنية، ويتم تقييم أدائهم باستمرار."
   Source: company_faq
8. "Question: هل توفرون خدمة التوعية الصحية عبر وسائل التواصل الاجتماعي؟
Answer: نعم، ننشر محتوى توعوياً بشكل منتظم على صفحاتنا الرسمية في وسائل التواصل الاجتماعي."
   Source: company_faq

### Query 3: "ما هي أوقات العمل"

**sentence-transformers/paraphrase-multilingual-mpnet-base-v2:**

*Vector Retriever (Time: 0.028s):*
1. "Question: ما هي أوقات عمل الشركة؟
Answer: أيام العمل من الأحد إلى الخميس، من 9 صباحًا حتى 9 مساءً."
   Source: company_faq
2. "نحن ملتزمون بتوفير خدماتنا لكم بأقصى درجات المرونة، مع مراعاة أوقات الراحة اللازمة لفريقنا:  
* **أيام العمل:** من الأحد إلى الخميس.
* **الساعات:** 9 صباحًا حتى 9 مساءً بتوقيت المملكة العربية السعودية (EEST).
* **العطلات الأسبوعية:** الجمعة والسبت.
* **العطل الرسمية:** بالإضافة إلى العطلات الأسبوعية..."
   Source: company_info
3. "Question: هل تتوفر خدمة حجز المواعيد على مدار 24 ساعة؟
Answer: نعم، نظام حجز المواعيد متاح على مدار 24 ساعة، ولكن الاستشارات نفسها تكون خلال ساعات العمل المحددة."
   Source: company_faq
4. "Question: هل يمكنني التواصل مع خدمة العملاء خارج أوقات العمل؟
Answer: الدردشة المباشرة متوفرة 24/7، أما الدعم الهاتفي والبريد الإلكتروني فيكون خلال ساعات العمل المحددة."
   Source: company_faq
5. "Question: ما هي أيام العطل الرسمية للشركة؟
Answer: الجمعة والسبت، بالإضافة إلى الأعياد الرسمية المعلنة في المملكة العربية السعودية."
   Source: company_faq

*Hybrid Retriever (Time: 0.028s):*
1. "Question: ما هي أوقات عمل الشركة؟
Answer: أيام العمل من الأحد إلى الخميس، من 9 صباحًا حتى 9 مساءً."
   Source: company_faq
2. "نحن ملتزمون بتوفير خدماتنا لكم بأقصى درجات المرونة، مع مراعاة أوقات الراحة اللازمة لفريقنا:  
* **أيام العمل:** من الأحد إلى الخميس.
* **الساعات:** 9 صباحًا حتى 9 مساءً بتوقيت المملكة العربية السعودية (EEST).
* **العطلات الأسبوعية:** الجمعة والسبت.
* **العطل الرسمية:** بالإضافة إلى العطلات الأسبوعية..."
   Source: company_info
3. "Question: هل يمكنني التواصل مع خدمة العملاء خارج أوقات العمل؟
Answer: الدردشة المباشرة متوفرة 24/7، أما الدعم الهاتفي والبريد الإلكتروني فيكون خلال ساعات العمل المحددة."
   Source: company_faq
4. "Question: هل تتوفر خدمة حجز المواعيد على مدار 24 ساعة؟
Answer: نعم، نظام حجز المواعيد متاح على مدار 24 ساعة، ولكن الاستشارات نفسها تكون خلال ساعات العمل المحددة."
   Source: company_faq
5. "Question: ما هي أيام العطل الرسمية للشركة؟
Answer: الجمعة والسبت، بالإضافة إلى الأعياد الرسمية المعلنة في المملكة العربية السعودية."
   Source: company_faq
6. "Question: ما هي القيم الأساسية للشركة؟
Answer: قيمنا الأساسية هي السرية والخصوصية، الجودة والاحترافية، التيسير والسرعة، والموثوقية والشفافية."
   Source: company_faq
7. "Question: ما هي رسالة الشفاء الرقمية؟
Answer: رسالتنا هي تقديم خدمات طبية موثوقة، آمنة، وسهلة الوصول لجميع أفراد المجتمع من خلال التكنولوجيا الحديثة."
   Source: company_faq

**intfloat/multilingual-e5-small:**

*Vector Retriever (Time: 0.013s):*
1. "Question: ما هي أوقات عمل الشركة؟
Answer: أيام العمل من الأحد إلى الخميس، من 9 صباحًا حتى 9 مساءً."
   Source: company_faq
2. "نحن ملتزمون بتوفير خدماتنا لكم بأقصى درجات المرونة، مع مراعاة أوقات الراحة اللازمة لفريقنا:  
* **أيام العمل:** من الأحد إلى الخميس.
* **الساعات:** 9 صباحًا حتى 9 مساءً بتوقيت المملكة العربية السعودية (EEST).
* **العطلات الأسبوعية:** الجمعة والسبت.
* **العطل الرسمية:** بالإضافة إلى العطلات الأسبوعية..."
   Source: company_info
3. "Question: هل يمكنني التواصل مع خدمة العملاء خارج أوقات العمل؟
Answer: الدردشة المباشرة متوفرة 24/7، أما الدعم الهاتفي والبريد الإلكتروني فيكون خلال ساعات العمل المحددة."
   Source: company_faq
4. "Question: هل تتوفر خدمة حجز المواعيد على مدار 24 ساعة؟
Answer: نعم، نظام حجز المواعيد متاح على مدار 24 ساعة، ولكن الاستشارات نفسها تكون خلال ساعات العمل المحددة."
   Source: company_faq
5. "Question: ما هي أيام العطل الرسمية للشركة؟
Answer: الجمعة والسبت، بالإضافة إلى الأعياد الرسمية المعلنة في المملكة العربية السعودية."
   Source: company_faq

*Hybrid Retriever (Time: 0.013s):*
1. "Question: ما هي أوقات عمل الشركة؟
Answer: أيام العمل من الأحد إلى الخميس، من 9 صباحًا حتى 9 مساءً."
   Source: company_faq
2. "Question: هل يمكنني التواصل مع خدمة العملاء خارج أوقات العمل؟
Answer: الدردشة المباشرة متوفرة 24/7، أما الدعم الهاتفي والبريد الإلكتروني فيكون خلال ساعات العمل المحددة."
   Source: company_faq
3. "نحن ملتزمون بتوفير خدماتنا لكم بأقصى درجات المرونة، مع مراعاة أوقات الراحة اللازمة لفريقنا:  
* **أيام العمل:** من الأحد إلى الخميس.
* **الساعات:** 9 صباحًا حتى 9 مساءً بتوقيت المملكة العربية السعودية (EEST).
* **العطلات الأسبوعية:** الجمعة والسبت.
* **العطل الرسمية:** بالإضافة إلى العطلات الأسبوعية..."
   Source: company_info
4. "Question: هل تتوفر خدمة حجز المواعيد على مدار 24 ساعة؟
Answer: نعم، نظام حجز المواعيد متاح على مدار 24 ساعة، ولكن الاستشارات نفسها تكون خلال ساعات العمل المحددة."
   Source: company_faq
5. "Question: ما هي أيام العطل الرسمية للشركة؟
Answer: الجمعة والسبت، بالإضافة إلى الأعياد الرسمية المعلنة في المملكة العربية السعودية."
   Source: company_faq
6. "Question: ما هي القيم الأساسية للشركة؟
Answer: قيمنا الأساسية هي السرية والخصوصية، الجودة والاحترافية، التيسير والسرعة، والموثوقية والشفافية."
   Source: company_faq
7. "Question: ما هي رسالة الشفاء الرقمية؟
Answer: رسالتنا هي تقديم خدمات طبية موثوقة، آمنة، وسهلة الوصول لجميع أفراد المجتمع من خلال التكنولوجيا الحديثة."
   Source: company_faq

**ibm-granite/granite-embedding-107m-multilingual:**

*Vector Retriever (Time: 0.007s):*
1. "Question: ما هي أوقات عمل الشركة؟
Answer: أيام العمل من الأحد إلى الخميس، من 9 صباحًا حتى 9 مساءً."
   Source: company_faq
2. "نحن ملتزمون بتوفير خدماتنا لكم بأقصى درجات المرونة، مع مراعاة أوقات الراحة اللازمة لفريقنا:  
* **أيام العمل:** من الأحد إلى الخميس.
* **الساعات:** 9 صباحًا حتى 9 مساءً بتوقيت المملكة العربية السعودية (EEST).
* **العطلات الأسبوعية:** الجمعة والسبت.
* **العطل الرسمية:** بالإضافة إلى العطلات الأسبوعية..."
   Source: company_info
3. "Question: هل تتوفر باقات اشتراك شهرية أو سنوية؟
Answer: نعم، نوفر باقات اشتراك شهرية وسنوية لتلبية الاحتياجات المختلفة، مع مزايا وخصومات خاصة."
   Source: company_faq
4. "Question: هل لديكم برنامج ولاء للعملاء الدائمين؟
Answer: قد نقدم برامج ولاء أو خصومات خاصة للعملاء الدائمين، يرجى متابعة إعلاناتنا."
   Source: company_faq
5. "Question: هل يمكنني التواصل مع خدمة العملاء خارج أوقات العمل؟
Answer: الدردشة المباشرة متوفرة 24/7، أما الدعم الهاتفي والبريد الإلكتروني فيكون خلال ساعات العمل المحددة."
   Source: company_faq

*Hybrid Retriever (Time: 0.006s):*
1. "Question: ما هي أوقات عمل الشركة؟
Answer: أيام العمل من الأحد إلى الخميس، من 9 صباحًا حتى 9 مساءً."
   Source: company_faq
2. "نحن ملتزمون بتوفير خدماتنا لكم بأقصى درجات المرونة، مع مراعاة أوقات الراحة اللازمة لفريقنا:  
* **أيام العمل:** من الأحد إلى الخميس.
* **الساعات:** 9 صباحًا حتى 9 مساءً بتوقيت المملكة العربية السعودية (EEST).
* **العطلات الأسبوعية:** الجمعة والسبت.
* **العطل الرسمية:** بالإضافة إلى العطلات الأسبوعية..."
   Source: company_info
3. "Question: هل يمكنني التواصل مع خدمة العملاء خارج أوقات العمل؟
Answer: الدردشة المباشرة متوفرة 24/7، أما الدعم الهاتفي والبريد الإلكتروني فيكون خلال ساعات العمل المحددة."
   Source: company_faq
4. "Question: هل تتوفر باقات اشتراك شهرية أو سنوية؟
Answer: نعم، نوفر باقات اشتراك شهرية وسنوية لتلبية الاحتياجات المختلفة، مع مزايا وخصومات خاصة."
   Source: company_faq
5. "Question: هل لديكم برنامج ولاء للعملاء الدائمين؟
Answer: قد نقدم برامج ولاء أو خصومات خاصة للعملاء الدائمين، يرجى متابعة إعلاناتنا."
   Source: company_faq
6. "Question: ما هي القيم الأساسية للشركة؟
Answer: قيمنا الأساسية هي السرية والخصوصية، الجودة والاحترافية، التيسير والسرعة، والموثوقية والشفافية."
   Source: company_faq
7. "Question: ما هي رسالة الشفاء الرقمية؟
Answer: رسالتنا هي تقديم خدمات طبية موثوقة، آمنة، وسهلة الوصول لجميع أفراد المجتمع من خلال التكنولوجيا الحديثة."
   Source: company_faq

## Performance Summary:


### sentence-transformers/paraphrase-multilingual-mpnet-base-v2
- **Total Processing Time:** 9.55 seconds
- **Vector Retriever Average Time:** 0.030 seconds per query
- **Hybrid Retriever Average Time:** 0.029 seconds per query

### intfloat/multilingual-e5-small
- **Total Processing Time:** 7.51 seconds
- **Vector Retriever Average Time:** 0.014 seconds per query
- **Hybrid Retriever Average Time:** 0.015 seconds per query

### ibm-granite/granite-embedding-107m-multilingual
- **Total Processing Time:** 6.87 seconds
- **Vector Retriever Average Time:** 0.008 seconds per query
- **Hybrid Retriever Average Time:** 0.007 seconds per query


## Evaluation Criteria:
1. **Performance Speed:** Faster retrieval times
2. **Document Relevance:** How well retrieved documents match the query intent
3. **Multilingual Support:** Better handling of Arabic queries
4. **Result Quality:** Accuracy and relevance of retrieved documents
5. **Consistency:** Stable performance and relevance across different queries

## What I Need From You:
1. **Rank the Embedding Models:** (1st, 2nd, 3rd) with justification based on both speed AND document relevance
2. **Compare Retriever Types:** Which is better - Vector or Hybrid? Consider both speed and relevance
3. **Document Relevance Analysis:** For each query, which model+retriever combination returned the most relevant documents?
4. **Final Recommendation:** Best combination (embedding model + retriever type) considering all factors
5. **Detailed Analysis:** Strengths and weaknesses of each combination in terms of:
   - Speed performance
   - Document relevance quality
   - Consistency across queries
6. **Production Considerations:** Recommendations for production environment usage

## Objective:
Select the best combination that provides the optimal balance between:
- Response speed
- Quality and relevance of retrieved results
- Efficiency in processing Arabic text
- Performance stability
- Document-query matching accuracy

## Additional Context:
- The system will be used for company information retrieval
- Arabic language support is crucial
- Both speed and accuracy are important
- The system should return highly relevant documents
- Consider scalability for production use

## Analysis Instructions:
Please analyze each query's results and evaluate:
1. Which retrieved documents are most relevant to each query
2. Which model consistently returns better relevant documents
3. Whether vector or hybrid retrieval provides better document relevance
4. The trade-off between speed and relevance quality

Please structure your response with clear sections for:
1. Document Relevance Analysis by Query
2. Speed Performance Analysis
3. Overall Model Ranking
4. Retriever Type Comparison
5. Final Recommendation with Justification


In [ ]:

# Initialize ChatOpenAI
try:
    llm = ChatOpenAI(
        model="gpt-4o",
        temperature=0,
        api_key=os.getenv("OPENAI_API_KEY"),
        base_url=os.getenv("BASE_URL")
    )

    print("🤖 Sending results to GPT-4o for expert evaluation...")
    print("-" * 60)

    # Get LLM evaluation
    llm_response = llm.invoke(llm_prompt)

    print("🎯 GPT-4o Expert Evaluation:")
    print("=" * 60)
    display(Markdown(llm_response.content))

except Exception as e:
    print(f"❌ Error with LLM evaluation: {str(e)}")
    print("💡 Please check your OpenAI API key and connection")



🤖 Sending results to GPT-4o for expert evaluation...
------------------------------------------------------------


### 5. Create Vector Store with "intfloat/multilingual-e5-small" and Save it in "vectorstore/"











In [ ]:
model_name = "intfloat/multilingual-e5-small"

persist_dir = "../vectorstore/"

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={'device' : 'cpu'},
    encode_kwargs={'normalize_embeddings':True}
)

# Create vector store
vector_store = FAISS.from_documents(company_chunks, embeddings)

# Save the vector store
vector_store.save_local(persist_dir)
print("💾 Vector store saved successfully")

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: intfloat/multilingual-e5-small


💾 Saving vector store...


In [34]:
print("📂 Loading vector store...")
vector_store = FAISS.load_local(persist_dir, embeddings, allow_dangerous_deserialization=True)
    
k = 5

# Create vector retriever
print("🔍 Creating vector retriever...")
vector_retriever = vector_store.as_retriever(search_kwargs={"k": k})

# Create BM25 retriever
print("📝 Creating BM25 retriever...")
bm25_retriever = BM25Retriever.from_documents(company_chunks)
bm25_retriever.k = k

# Create hybrid retriever
print("🔄 Creating hybrid retriever...")
hybrid_retriever = EnsembleRetriever(
       retrievers=[bm25_retriever, vector_retriever],
       weights=[0.2, 0.8]
   )

print("✅ Retrievers created and hybrid retriever is ready.")

📂 Loading vector store...
🔍 Creating vector retriever...
📝 Creating BM25 retriever...
🔄 Creating hybrid retriever...
✅ Retrievers created and hybrid retriever is ready.


In [37]:
query = "كيف يمكنني التواصل معكم"
retrieved_docs = hybrid_retriever.get_relevant_documents(query)
for i, doc in enumerate(retrieved_docs, 1):
    print(f"--- Document {i} ---")
    print("Metadata:", doc.metadata)
    print("Content:", doc.page_content)
    print()


--- Document 1 ---
Metadata: {'source': 'company_faq', 'type': 'faq', 'doc_id': '5'}
Content: Question: كيف يمكنني التواصل مع الدعم الفني؟
Answer: يمكنك التواصل مع الدعم الفني عبر الدردشة المباشرة، الهاتف (9200-000-000)، أو البريد الإلكتروني (support@alshifa-care.com).

--- Document 2 ---
Metadata: {'source': 'company_faq', 'type': 'faq', 'doc_id': '14'}
Content: Question: كيف يمكنني إلغاء الاشتراك؟
Answer: يمكنك تسجيل الدخول إلى حسابك والتوجه إلى "إعدادات الحساب" ثم "إلغاء الاشتراك"، أو التواصل مع الدعم الفني.

--- Document 3 ---
Metadata: {'source': 'company_faq', 'type': 'faq', 'doc_id': '72'}
Content: Question: هل يمكنني التواصل مع الطبيب بعد انتهاء الاستشارة؟
Answer: يمكن التواصل مع الطبيب للمتابعة ضمن استشارة جديدة، أو للاستفسارات القصيرة عبر خاصية الرسائل إن كانت متاحة في باقتك.

--- Document 4 ---
Metadata: {'company_title': 'شركة الشفاء الرقمية للرعاية الصحية', 'section': '📞 معلومات الاتصال', 'source': 'company_info', 'type': 'general_info', 'filename': 'info.md', 'doc_id': 'c